# Boxplots

## Script setup

In [ ]:
# Third-party and standard modules
from typing import Dict, Hashable
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import pytz
from importlib import reload
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
# User-defined modules.
# The reload() function is needed to update modules after
# changes are made to their files.
from modules import dataframe_manip as dfm
from modules import misc, schemas, College1AcPtTimeSeries
# The reload() function is needed to update modules after
# changes are made to their files.
dfm = reload(dfm)
misc = reload(misc)
schemas = reload(schemas)
College1AcPtTimeSeries = reload(College1AcPtTimeSeries)

In [ ]:
schema: schemas.AcPtTimeSeries =\
    College1AcPtTimeSeries.College1AcPtTimeSeries

## Script parameters

The following values are this script's parameters.

In [ ]:
#filepath = './cisco_wifi_data_until_20190513.csv'
filepath = './wifi_data_until_20190204.csv'
timezone = pytz.timezone('US/Pacific')
'''buildings = [
                'SCC', 'HAHN', 'CROOKSHANK', 'THATHCHER', 'CLARK',
                'CARNEGIE', 'MASON', 'PEARSON', 'ALEXANDER', 'ITB'
]
'''
buildings = [
                'ALEXANDER'
]

#Fall Semester 
start_date = '2018-11-21' #None
end_date = '2018-11-23' #None


boxplot_hourly_average = True
holiday_analysis = True

## Data Processing

In [ ]:
# A time-series dataframe holding occupancy counts
# for many access points (columns).
occ: pd.DataFrame = dfm.csv_to_timeseries_df(
    filepath=filepath, timezone=timezone
)

In [ ]:
if start_date is not None: 
    if end_date is None:
        occ = occ.loc[start_date:]
    else:
        occ = occ.loc[start_date:end_date]
else:
    if end_date is not None:
        occ = occ.loc[:end_date]

In [ ]:
# Eliminate columns for access points not in the buildings we want.
occ = occ.drop(columns=list(filter(
    # add to drop-list if NOT in the buildings we want.
    lambda col: schema.col_to_building(col, safe=True) not in buildings,
    occ.columns.to_list()
)))
'''
# We must assume that intervening na-values are 0's.
occ = dfm.fill_intervening_nas(
    occ, inplace=True, fill_val=0
)
'''

In [ ]:
occ_bybuild = occ.groupby(
    by=schema.col_to_building, axis=1
)

In [ ]:
print('buildings not found: {}'.format(set(buildings) - set(occ_bybuild.groups.keys())))

# We can only work with the buildings we found.
buildings = list(occ_bybuild.groups.keys())

## Boxplots

In [ ]:
if(boxplot_hourly_average == True):
    means_by_building: Dict[Hashable, pd.DataFrame] = {}
    maxmean = float('-inf')
    for build in buildings:
        means_by_building[build] = dfm.row_means(occ_bybuild.get_group(build))
        newmax = means_by_building[build].max()
        newmin = means_by_building[build].min()
        maxmean = newmax if newmax > maxmean else maxmean
    #print(maxmean)

    for build in buildings:
        mean_occ: pd.DataFrame = dfm.row_means(occ_bybuild.get_group(build))

        fig, ax = plt.subplots(figsize=(24, 15))
        ax.set_title(build)
        ax.set_xlabel('hour of the day')
        ax.set_ylabel('mean number of connected devices')
        ax.set_ylim(0, 200)

        seaborn.boxplot(
            # The hours (a number [0,23]) for each row.
            x=mean_occ.index.hour,
            # The total occupancy at that hour.
            y=mean_occ,
            ax=ax
        )


        fig, ax = plt.subplots(figsize=(24, 15))
        ax.set_title(build + ' (weekdays)')
        ax.set_xlabel('hour of the day')
        ax.set_ylabel('mean number of connected devices')
        ax.set_ylim(0, maxmean)

        wkdays = mean_occ.loc[mean_occ.index.dayofweek<5].copy()

        seaborn.boxplot(
            # The hours (a number [0,23]) for each row.
            x=wkdays.index.hour,
            # The total occupancy at that hour.
            y=wkdays,
            ax=ax
        )
        fig.savefig('./{}.png'.format(build))
        '''
        fig, ax = plt.subplots(figsize=(24, 15))
        ax.set_title(build + ' (weekends)')
        ax.set_xlabel('hour of the day')
        ax.set_ylabel('mean number of connected devices')
        ax.set_ylim(0, maxmean)

        wkends = mean_occ.loc[mean_occ.index.dayofweek>=5].copy()

        seaborn.boxplot(
            # The hours (a number [0,23]) for each row.
            x=wkends.index.hour,
            # The total occupancy at that hour.
            y=wkends,
            ax=ax
        )
        '''

## Holidays

In [ ]:
if (holiday_analysis == True):
    holi_caldr = calendar()
    holidays = holi_caldr.holidays(start = start_date, end = end_date)

    display(holidays)
    holiday_ts = pd.to_datetime(holidays)

    truncated_time = occ.index.normalize()
    occ['time_no_timezone'] = truncated_time.tz_localize(None)
    occ['isHoliday'] = occ['time_no_timezone'].isin(holiday_ts)

    #print(occ['isHoliday'].any())
    #print(type(occ.index))
    display(occ)

In [ ]:
#occ.grouby('isHoliday').size()
#occ['2018-11-22']

In [ ]:
if (holiday_analysis == True):
    holiday_occ = occ.loc[occ['isHoliday'] == True]
    #display(occ)
    holiday_occ = holiday_occ.drop(['time_no_timezone', 'isHoliday'], axis =1)
    occ_holiday_bybuild = holiday_occ.groupby(
        by=schema.col_to_building, axis=1
    )

    means_by_building: Dict[Hashable, pd.DataFrame] = {}
    maxmean = float('-inf')
    for build in buildings:
        means_by_building[build] = dfm.row_means(occ_holiday_bybuild.get_group(build))
        newmax = means_by_building[build].max()
        newmin = means_by_building[build].min()
        maxmean = newmax if newmax > maxmean else maxmean
    print(maxmean)

    for build in buildings:
        mean_occ_holiday: pd.DataFrame = dfm.row_means(occ_holiday_bybuild.get_group(build))

        fig, ax = plt.subplots(figsize=(24, 15))
        ax.set_title(build + ' (holidays from ' + start_date + ' to ' + end_date +')')
        ax.set_xlabel('hour of the day')
        ax.set_ylabel('mean number of connected devices')
        ax.set_ylim(0, 200)

        seaborn.boxplot(
            # The hours (a number [0,23]) for each row.
            x=mean_occ_holiday.index.hour,
            # The total occupancy at that hour.
            y=mean_occ_holiday,
            ax=ax
        )
  

In [ ]:
#holiday_occ